# Import Library

In [1]:
import os
import requests
import json
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor
from typing import Optional
from pydantic import BaseModel, Field, field_validator, root_validator
from langchain.output_parsers import PydanticOutputParser


In [2]:
load_dotenv()

True

In [3]:
GROQ_API_KEY = os.getenv("GROK_API_KEY")
SECTORS_API_KEY = os.getenv("SECTOR_API_KEY")

def retrieve_from_endpoint(url: str) -> dict:
    headers = {"Authorization": SECTORS_API_KEY}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)


# Coding

In [4]:
class Stock(BaseModel):
    """Information about a company's stock"""

    symbol: str = Field(description="The stock symbol")
    name: str = Field(description="The name of the company for which the stock symbol represents")
    sector: Optional[str] = Field(default=None, description="The sector of the company")
    industry: Optional[str] = Field(default=None, description="The industry of the company")
    market_cap: Optional[int] = Field(default=None, description="The market capitalization of the company")
    
    @field_validator("symbol", mode="before")
    @classmethod
    def validate_symbol_4_letters(cls, values: dict) -> dict:
        symbol = values["symbol"]
        
        if len(symbol) != 4:
            raise ValueError("Symbol must be 4 letters long")
        return values

    # Validate that the market cap is a positive number (if provided)
    @root_validator(pre=True)
    @classmethod
    def validate_market_cap(cls, values: dict) -> dict:
        market_cap = values["market_cap"]
        
        if market_cap < 0:
            raise ValueError("Market cap must be a positive number")
        return values   

C:\Users\Arseno Feri Alzahabi\AppData\Local\Temp\ipykernel_41448\4275741177.py:20: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @root_validator(pre=True)


In [5]:
parser = PydanticOutputParser(pydantic_object=Stock)

In [6]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Information about a company\'s stock", "properties": {"symbol": {"description": "The stock symbol", "title": "Symbol", "type": "string"}, "name": {"description": "The name of the company for which the stock symbol represents", "title": "Name", "type": "string"}, "sector": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "The sector of the company", "title": "Sector"}, "industry": {"anyOf": [{"type": "string"}, {"type": "null"}], "default": null, "description": "The industry of the compa

In [7]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-groq-70b-8192-tool-use-preview",
    groq_api_key=GROQ_API_KEY,
)

structured_llm = llm.with_structured_output(Stock)
print(type(structured_llm))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [8]:
@tool

def get_company_overview(stock:str)-> str:
    """
    Get company overview from sector API
    """

    url = f"https://api.sectors.app/v1/company/report/{stock}/?sections=overview"
    
    return retrieve_from_endpoint(url)

In [9]:
@tool
def get_today_date()->str:
    """
    Get today's date
    """
    return datetime.datetime.now().date().strftime("%Y-%m-%d")

In [10]:
@tool
def get_top_companies_by_tx_volume(start_date: str, end_date: str, top_n: int = 5) -> str:
    """
    Get top companies by transaction volume
    """
    url = f"https://api.sectors.app/v1/most-traded/?start={start_date}&end={end_date}&n_stock={top_n}"

    return retrieve_from_endpoint(url)

In [11]:
@tool
def get_daily_tx(stock: str, start_date: str, end_date: str) -> str:
    """
    Get daily transaction for a stock
    """
    url = f"https://api.sectors.app/v1/daily/{stock}/?start={start_date}&end={end_date}"

    return retrieve_from_endpoint(url)

In [12]:
tools = [
    get_company_overview,
    get_top_companies_by_tx_volume,
    get_daily_tx,
    get_today_date
]

In [13]:
text = """
    Bank Central Asia (BCA) is a bank in Indonesia and is part of the finance sector.
    It is in the banking industry and has a market capitalization of $85 billion.
    It trades under the symbol BBCA on the Indonesia Stock Exchange.
    """
out = structured_llm.invoke(text)

TypeError: string indices must be integers, not 'str'

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Provide detailed stock information in a structured format."),
    ("human", "give me information for the stock: {symbol} in indonesia stock market"),
    MessagesPlaceholder("agent_scratchpad")  # Required for intermediate steps
])


In [ ]:
import pydantic
print(pydantic.__version__)



2.9.2


In [ ]:
create = "Create a fictitious company called Supertype Inc. that is in the technology sector and has a market capitalization of $10 million. It trades under the symbol SUPA."
generated = structured_llm.invoke(create)
print(generated)
# Stock(symbol='SUPA', name='Supertype Inc.', sector='technology', industry=None, market_cap=10000000)


symbol='SUPA' name='Supertype Inc.' sector='technology' industry=None market_cap=10000000
